# BTC and ETH address generators
Compare different Python implementations of the address generation for BTC and ETH.
Make a useful tool to generate beautiful vanity addresses.

### Utilities

In [ ]:
# convert private key string to the hex

def hex_key_string_to_bytes(private_key_hex):
    # Remove '0x' prefix if present
    if private_key_hex.startswith('0x') or private_key_hex.startswith('0X'):
        private_key_hex = private_key_hex[2:]

    private_key_bytes = bytes.fromhex(private_key_hex)
    if len(private_key_bytes) != 32:
        raise ValueError("Private key must be 32 bytes long")

    return private_key_bytes

### ETH address generator implementation 1

In [ ]:
# conda install eth-utils -c conda-forge

from ecdsa import SigningKey, SECP256k1
from eth_utils import keccak, to_checksum_address

def eth_address(private_key_hex):

    private_key_bytes = hex_key_string_to_bytes(private_key_hex)

    # Generate public key
    sk = SigningKey.from_string(private_key_bytes, curve=SECP256k1)
    vk = sk.get_verifying_key()

    # Get the public key in uncompressed format (64 bytes)
    public_key_bytes = vk.to_string()

    # Keccak-256 hash of public key
    public_key_hash = keccak(public_key_bytes)

    # Take last 20 bytes to form the address
    address_bytes = public_key_hash[-20:]
    address_hex = '0x' + address_bytes.hex()

    # Convert to checksummed address (EIP-55)
    checksum_address = to_checksum_address(address_hex)
    return checksum_address

### ETH address generator implementation 2

In [ ]:
# conda install bip-utils

from bip_utils import Bip44, Bip44Coins

def eth_address(private_key_hex):
    private_key_bytes = hex_key_string_to_bytes(private_key_hex)
    return Bip44.FromPrivateKey(private_key_bytes, Bip44Coins.ETHEREUM).PublicKey().ToAddress()

### ETH address generator implementation 3

In [ ]:
# conda install bip-utils

from bip_utils import Secp256k1PrivateKey, EthAddr, P2WPKHAddr, WifEncoder

def eth_address(private_key_hex):
    
    private_key_bytes = hex_key_string_to_bytes(private_key_hex)
    # Create a Secp256k1PrivateKey object from the private key bytes
    private_key = Secp256k1PrivateKey.FromBytes(private_key_bytes)
    
    # Get the public key from the private key
    public_key = private_key.PublicKey()
    
    # Ethereum Address Generation
    eth_address = EthAddr.EncodeKey(public_key)
    return eth_address

### ETH addr generator usage example

In [ ]:
private_key = '01b712d2fb6a68c6a6224d49eeedfacb58594d57f91359d81d7a2a383f262649'
address = eth_address(private_key)
print(f"Ethereum Address: {address}")

---

## BTC BIP84 Native Segwit (P2WPKH) address generator
(Because why do you need an old legacy address?)

### BTC address from hex string, implementation 1

In [ ]:
# conda install bip-utils

from bip_utils import Secp256k1PrivateKey, EthAddr, P2WPKHAddr, WifEncoder

def btc_address(private_key_hex):
    # Bitcoin Native SegWit Address
    private_key_bytes = hex_key_string_to_bytes(private_key_hex)
    # Create a Secp256k1PrivateKey object from the private key bytes
    private_key = Secp256k1PrivateKey.FromBytes(private_key_bytes)
    
    # Get the public key from the private key
    public_key = private_key.PublicKey()
    
    # Generate Bitcoin Native Segwit (P2WPKH) address
    btc_address = P2WPKHAddr.EncodeKey(public_key, hrp='bc')
    return btc_address

### BTC address from hex string, implementation 2

In [ ]:
from bip_utils import Bip84, Bip84Coins

def btc_address(private_key_hex):
    # Bitcoin Native SegWit Address
    private_key_bytes = hex_key_string_to_bytes(private_key_hex)
    bip84 = Bip84.FromPrivateKey(private_key_bytes, Bip84Coins.BITCOIN)
    btc_address = bip84.PublicKey().ToAddress()
    # wif_private_key = bip84.PrivateKey().ToWif()
    # print(wif_private_key)
    return btc_address

### BTC address from WIF string

In [ ]:
from bip_utils import WifDecoder

def btc_address_WIF(wif_private_key):
    priv_key_bytes, _ = WifDecoder.Decode(wif_private_key)
    priv_key_hex = priv_key_bytes.hex()
    return btc_address(priv_key_hex)

In [ ]:
def Wif_from_priv(private_key_hex):
    private_key_bytes = hex_key_string_to_bytes(private_key_hex)
    bip84 = Bip84.FromPrivateKey(private_key_bytes, Bip84Coins.BITCOIN)
    wif_private_key = bip84.PrivateKey().ToWif()
    return wif_private_key    

### BTC addr generator usage example

In [ ]:
wif_private_key = 'L3L91g1K5TWq3G8nWvV7c8h3sqkv6TecBoHv4zEApZLaqoWfNHsf'
btc_address_WIF(wif_private_key)

---

## Test Data

In [ ]:
test_eth_addr = [
    ['01b712d2fb6a68c6a6224d49eeedfacb58594d57f91359d81d7a2a383f262649', '0x0004Aa53eD3CD4E1d9B6C77BFA697B263A584d66'],
    ['7ed8ed33c677394789289ec94d64e2a92ae13ae7b2361eaa7d6e940ae5180698', '0x000966e72E852FB9ed4F97B46B7Eb02A55fdcB29'],
    ['6b025864ed1093b13854a866d546db18f63128d989e68a2efd6f4e30e080b2d1', '0x5Ec5a8cbBa5d72aaea94153CEce78dbAC74e6888'],
    ['6c881c1facdc443a20c0e178c94d0a015904d8efcd58330ea2ac46861a0a8ad1', '0x1111aAFB8CD27f09dF48Ec7Dc730700147E4E5C5'],
    ['394e583f12ec94195dc209d441bd3076e8ebb49fa4188995741214e172e48988', '0x2222efC7fc5e759125687bC01772c63D3975398f'],
    ['162cf5d7bebf4e565249972ab93f3cb24911a2c188e7fb5e9956fd3e5a7ba114', '0x3333aa269816b396335EE97e07FeE72A712ee4aC'],
    ['b147ca80d4622a1579ecd5f38a75233a63238fefec86f20615d3ef7d7de90580', '0xa888fA2433106E33c88c88644c98844cf98eB8EA'],
    ['668fd7add0cc6766182ffd9a2e0735629be928e8d6f2f2de0f0c7cc9d0b425ea', '0x988BCF8737887a5a8888327499D54356495b9818'],
]  
 

In [ ]:
test_btc_addr = [
    ['KzFAqBvM3hDpSK4bMWt9nCV7DbAPx4cGcg5ftQ7gf5jedLy2RdDa', 'bc1qz92wjad4y3fxdg7q8xzquarc3kzskc24z5js8s'],
    ['L31JaGQre72BzJFodMhJcam2LAU5TahtEC3Kk2vRapxw6TdYZDNu', 'bc1q9x3ta6grdqp79n3r5329uldhunpueq5llew0n7'],
    ['L25VrNZTH7j6F9ShK18xUGRBEyoCcxKACKsADySpLgC4vs9vLbTB', 'bc1qte3j3qdc59sjpwg2a7jfyjjl6qw0229qrwel2j'],
    ['KzgrHjKjUKNapmVVy4qy4auMC8DRuTpePKCfV3iV7QtMdcPvKTmY', 'bc1qnm3y0rjjgutu9qu8fpg05wtxkmr8wzr6ys0y8u'],
    ['KyvChFARYArJ9cM8KqKv3VLhiiddpWrCVh7j8hztWUYWFqgtf9zL', 'bc1q2t38xuevj8tfcljzmjhrsu69ac45sg59c6s0dq'],
    ['L3mrNfgHzdHpaSGfdaUZbDPPDt5AugXnRAxASE5pZABFrDS3UHTe', 'bc1q55jqpmgyjwtyu9ercxwz03udvwa85r4v7mzpsn'],
    ['L3hQA4b58N9Ndx4VgGFrtYLNGGrpPVhMa1BtbPKVD48JUiQskXZk', 'bc1qtest4ryd8xz84kzu4xyt8yn06ak24dn6nm763p'],
    ['L3yDTjp6QaAX7yjutZG8FcpTYeR8ELaDjTkFjv5vfHQ1TYoRsygt', 'bc1queryfduv2f2y7gstthq2pyqlt69qgzfxz7lxym'],
    ['L3L91g1K5TWq3G8nWvV7c8h3sqkv6TecBoHv4zEApZLaqoWfNHsf', 'bc1qc543kvt9t8glrw9xk2nas6a69yz3cglqrapcat'],
    ['L3eb6zxpKy7eEwKf7HsLkzJTfjfaF324TyHrhpvKL73MkJhQbCGe', 'bc1qmjsn3xrqsqrfeux6ky05mktj4yelu7jt8wgaay'],
]

In [ ]:
# run tests 
def run_test(test_data, testing_func):
    all_tests_passed = True
    for idx, (private_key, expected_address) in enumerate(test_data, start=1):
        generated_address = testing_func(private_key)
        test_passed = generated_address == expected_address
        if test_passed:
            print(f"Test {idx}: PASS")
        else:
            print(f"Test {idx}: FAIL")
            print(f"  Expected Address: {expected_address}")
            print(f"  Generated Address: {generated_address}")
            all_tests_passed = False
    
    if all_tests_passed:
        print("\nAll tests passed successfully!")
    else:
        print("\nSome tests failed. Please check the discrepancies above.")


In [ ]:
# run ETH tests
run_test(test_eth_addr, eth_address)

In [ ]:
# run BTC tests
run_test(test_btc_addr, btc_address_WIF)

---

## Vanity BTC address generator

In [ ]:
import secrets

def validate_bech32_chars(adr): 
    # invalid characters: A-Z, b, i, o, 1
    for c in adr:
        if c not in 'qpzry9x8gf2tvdw0s3jn54khce6mua7l': # Bech32 character set
            print('Invalid charaters: A-Z, b, i, o, 1')
            return False
    return True

run = True # in future this can stop separate threads

def vanity_btc_address(prefix, suffix):
    if not validate_bech32_chars(prefix) or not validate_bech32_chars(suffix):
        return
    n = 0
    while run:
        n += 1
        if n % 20000 == 0: print('.', end='')
        pk = secrets.token_hex(32)
        ad = btc_address(pk)
        adp = ad[4:] # do not count mandatory prefix 'bc1q'
        if adp.startswith(prefix): # and adp.endswith(suffix):
#        if adp.endswith(suffix):
            print(n)
            print(Wif_from_priv(pk), ad)
            break  

In [ ]:
run = True
for i in range(2):
    vanity_btc_address('ute', 'why')

## Vanity ETC address generator

In [ ]:
import secrets

def validate_eth_chars(adr): 
    for c in adr:
        if c not in '0123456789abcdefABCDEF': # ETH character set
            print('Invalid charaters, use only 0123456789abcdefABCDEF')
            return False
    return True

run = True # in future this can stop separate threads

def vanity_eth_address(vanity_func):
    global run
    n = 0
    while run:
        n += 1
        if n % 10000 == 0: print('.', end='')
        pk = secrets.token_hex(32)
        ad = eth_address(pk)
        adp = ad[2:] # do not count prefix 0x
        if vanity_func(adp):
            run = False
            print(n)
            print(pk, ad)
            break      

In [ ]:
def vanity_prefix_suffix(adp):
    if adp.startswith('88') and adp.endswith('88'): return True
    return False

# vanity_eth_address(vanity_prefix_suffix)

In [ ]:
def vanity_subset(adp):
    if set(adp) <= set('0123456789afeAFE'): return True
    return False

vanity_eth_address(vanity_subset)

In [ ]:
def vanity_count(adp):
    if adp.count('8') > 13: return True
    return False

# vanity_eth_address(vanity_count)

In [ ]:
def vanity_includes(adp):
    if '888' in adp: return True
    return False

vanity_eth_address(vanity_includes)

In [ ]:
def check_repeated_sequences(s):
    i = 0
    n = len(s)
    while i < n:
        c = s[i]
        count = 1
        i += 1
        while i < n and s[i] == c:
            count += 1
            i += 1
        if count < 2:
            return False
    return True

### Threaded execution

In [ ]:
import threading

threads = []
run = True
for n in range(2):
    thread = threading.Thread(target=vanity_eth_address, args=(vanity_prefix_suffix,))
    thread.start()
    threads.append(thread)

In [ ]:
# Wait for all threads to complete
for thread in threads:
    thread.join()